In [8]:
!pip install -U kiwipiepy PyMuPDF langchain_community langchain sentence-transformers faiss-gpu rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 198.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 168.0 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
from huggingface_hub import login
login(token="<Your Token>")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import unicodedata
import torch
import re
import pandas as pd
from tqdm import tqdm
import fitz
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from kiwipiepy import Kiwi
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from peft import PeftModel, PeftConfig

In [2]:
kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

def setup_embedding_models(model_path="BAAI/bge-multilingual-gemma2"):
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return embeddings
    
def process_pdf(file_path, chunk_size=800, chunk_overlap=200):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
        
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks

def create_vector_db(chunks, embeddings):
    """FAISS DB 생성"""
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)

    faiss_retriever = db.as_retriever()
    
    kiwi_bm25 = BM25Retriever.from_documents(chunks, preprocess_func=kiwi_tokenize)
    ensemble_retriever = EnsembleRetriever(
        retrievers=[kiwi_bm25, faiss_retriever],
        weights=[0.3, 0.7],
        search_type="mmr", 
        search_kwargs={"k": 5,}
    )
    return db, ensemble_retriever

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    embeddings = setup_embedding_models()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path) 
        db, ensemble_retriever = create_vector_db(chunks, embeddings=embeddings)
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': ensemble_retriever
        }
    return pdf_databases

### 폴더구조
- workspace
    - test_source [폴더]
    - train_source [폴더]
    - llama3.1_qlora [폴더]
    - train.csv
    - test.csv
    - sample_submission.csv
    - Model Finetuning.ipynb
    - RAG Pipeline.ipynb

In [3]:
base_directory = './' # pdf의 상위 폴더 경로
df = pd.read_csv('./test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:02<00:22,  2.79s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:33<00:14,  7.17s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:41<00:07,  7.19s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:49<00:00,  5.46s/it]


In [4]:
def setup_llm_pipeline(lora_path="./llama3.1_qlora/"):
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    peft_config = PeftConfig.from_pretrained(lora_path)
    base_model = AutoModelForCausalLM.from_pretrained(
        peft_config.base_model_name_or_path, # "meta-llama/Meta-Llama-3.1-70B-Instruct"
        quantization_config=bnb_config,
        device_map="auto"
    )

    model = PeftModel.from_pretrained(
        base_model, 
        lora_path,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    
    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)
    tokenizer.use_default_system_prompt = False

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.3,
        return_full_text=False,
        max_new_tokens=512
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

# LLM 파이프라인
llm = setup_llm_pipeline()

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

In [7]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    당신은 재정 정보에 대한 전문가 입니다. 주어진 질문에 올바른 답변을 하세요. 다음 지침을 엄격히 따르세요:
    
    1. 제공된 재정 정보만을 사용하여 정확하고 객관적으로 답변하세요.
    2. 재정 용어, 예산 항목, 정책 등에 대해 명확하게 설명하세요.
    3. 숫자, 통계, 예산 금액 등은 정확히 인용하고, 필요시 간단한 해석을 덧붙이세요.
    4. 정확한 화폐 단위를 사용하세요.
    5. 답변할 수 없는 질문에는 "제공된 정보로는 이 질문에 답변할 수 없습니다."라고 정직하게 말하세요.
    6. 재정 정책이나 예산 집행의 효과에 대해 개인적 의견을 제시하지 마세요.
    7. 필요한 경우, 관련 재정 보고서나 자료의 출처를 언급하세요.
    8. 복잡한 재정 개념은 일반 국민도 이해할 수 있도록 쉽게 설명하세요.
    9. 답변은 논리적으로 구성하고, 필요시 항목별로 나누어 제시하세요.
    10. 답변은 간결하고 직접적으로 제시하세요.
    11. 잘못된 답변을 하는 경우 당신은 벌점을 받게 됩니다.

    주어진 정보:
    {context}

    답변을 하는 말투는 아래 예시 답변 스타일을 따라하라.
    
    예시 답변 스타일)

    질문: 
    2024년 예산 기준 특별회계는 얼마이며, 중앙정부 총지출 대비 어느 정도의 비율을 차지하는가?
    
    답변:
    2024년 예산 기준 특별회계는 81.7조원으로 중앙정부 총지출의 12.4%를 차지한다.

    질문:
    2022년 국가 예산의 불용액은 몇 조원인가?

    답변:
    2022년 국가 예산의 불용액은 14.8조원이다.

    질문:
    2022년 결산 기준으로 사회보험성기금의 이월금 규모는 몇 억원인가요?

    답변:
    223억원입니다.

    질문:
    추가경정예산은 2008년 이후 몇 번의 추가경정예산이 편성되었는가?

    답변:
    16회

    질문:
    2024년에 교육재정교부금에서 유아교육비 및 보육료 지원에 할당된 비중은?

    답변:
    4.40%

    질문:
    이상기후로 인한 수해 관련 재해 대응을 위해 어떤 시설이 신규 설치되고 있으며, 그 목적은 무엇인가?

    답변:
    빗물저장시설(우수저류시설) 6개소와 하천 수위계 센서에 연계하여 자동으로 출입을 통제하는 시설이 전국 180개소에 구축 중이며, 이는 도심 저지대 침수피해를 예방하고 하상도로 이용자의 인명피해를 최소화하기 위한 목적으로 이루어지고 있다.

    질문:
    교육재정 교부금이란?

    답변:
    교육의 균형발전을 위해 교육기관 및 교육행정기관 경비를 국가가 교부하는 일반지원금으로, 일반회계 보통・특별교부금, 고교 무상교육 증액분, 특별회계 유아교육비・보육료 지원 교부금 등 4가지로 구성되었다.

    질문:
    지난 10년간(2010~2021년)의 세외수입은 GDP 대비 어떻게 변화하였으며, 이에 대한 어떤 분석이 제시되었는가?

    답변:
    지난 10년간(2010~2021년)의 세외수입은 GDP 대비 1.4%포인트 감소하였으며, 세입 증가와 비교하여 정부의 수입이 소폭 증가한 것으로 분석되었다.
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    질문: 
    {question}
    <|eot_id|>
    <|start_header_id|>model<|end_header_id|>
    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    full_response = full_response.replace("답변:", "").replace("model", "").replace("user", "").replace("system", "").replace('\n', ' ')
    full_response = re.sub(r'\s+', ' ', full_response)
    full_response = full_response.strip()
    
    print(f'Answer: {full_response}\n')

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:06<10:52,  6.73s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2조 3,000억원입니다.

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 2/98 [00:24<21:09, 13.22s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Answer: 국가결산보고서에서는 우발부채를 '우발사항'으로 공시하고 있으며, 지방자치단체 회계기준에서는 우발부채를 '우발손실'로 정의하고 있습니다.

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Answering Questions:  99%|█████████▉| 97/98 [16:37<00:13, 13.08s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: 우발부채는 미래에 특정 사건(들)이 일어나지 않는 한 발생하지 않는 의무로, 하나 또는 그 이상의 조건이 충족되어야 금융거래로 인식된다는 점에서 ‘부채’와 차이점이 있습니다. 우발부채는 재정위험(fiscal risk)을 발생시키는 구체적 원인이 되며, 의도적인 공공정책이나 재정위기와 같이 예기치 못한 사건으로 인해 발생할 수 있습니다. 따라서 우발부채 관리는 재정건전성, 재정위기 관리 등 다양한 관점에서 중요하게 다뤄져야 할 필요가 있습니다.

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


Answering Questions: 100%|██████████| 98/98 [17:07<00:00, 10.48s/it]

Answer: 보증(guarantees)은 명시적 우발부채의 가장 보편적 형태이며, 파생금융상품 형태의 보증이나 표준화된 보증 제도하에서 설정하는 충당부채 형태의 보증은 재정상태표에 부채로 기록된다. 표준화 보증(standardized guarantees)은 통상 아주 적은 금액에 대해 획일적 조건으로 대규모로 발행하는 보증으로, 수출(무역)신용 보증, 환보증, 다양한 종류의 보험(예금, 농작물, 자연재해 등), 농민융자, 모기지론, 학자금융자, 중소기업융자 등이 있다. 주로 유사한 특징을 지니고 위험 분산을 목적으로 하는 보증들이 많으며, 보증인은 확률통계에 기반하여 평균손실(채무불이행률)을 추정할 수 있다.



In [8]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./submission.csv", encoding='UTF-8-sig', index=False)